In [2]:
import meshio as mio
import meshplot as mp
import numpy as np
from utils import *
import torch
from tqdm import tqdm

In [3]:
# m = mio.read("../meshes/sphere/sphere3.msh")
m = mio.read("../meshes/armadillo/armadillop4.msh")

order = 3 if m.cells[-1].type == 'tetra20' else 4
T = m.cells[-1].data.astype(int)
V = m.points

In [17]:
def energy(order, uv, pts, target):
    img_p = gmapping(order, uv, pts)
    
    if type(pts) is np.array or type(pts) is np.ndarray:
        return np.linalg.norm(img_p - target)**2;
    else:
        return torch.norm(img_p - target)**2

In [18]:
def inver_gmapping(order, uv0, pts, target, eps=1e-10):
    tuv0 = torch.tensor(uv0, requires_grad=True, dtype = torch.float64)
    tpts = torch.tensor(pts, dtype = torch.float64)
    tt = torch.tensor(target, dtype = torch.float64)
    
    optimizer = torch.torch.optim.LBFGS([tuv0],
                              lr=0.1,
                              history_size=20,
                              max_iter=10,
                              line_search_fn="strong_wolfe")
    
    f = lambda uv: energy(order, uv, tpts, tt)
    vv = []
    
    for i in tqdm(range(1000)):
        optimizer.zero_grad()
        objective = f(tuv0)
        objective.backward()
        optimizer.step(lambda: f(tuv0))
        vv.append(objective.item())
    
        if objective.item() < eps:
            break
            
    
    return tuv0.detach().numpy(), vv

In [19]:
ti = 80
# 
rV, rT = sample_tet(8)

TT = T[ti, :]
pts = V[TT]*100

target = np.array([80., 80., 70.])

opt, vv = inver_gmapping(order, rV[0, :], pts, target)

  3%|▎         | 29/1000 [00:02<01:03, 15.39it/s]


In [20]:
print(type(opt), type(pts))

print(gmapping(order, opt, pts))
print(energy(order, opt, pts, target))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
[80.00000004 79.99999948 69.99999971]
3.6041587584165737e-13


In [ ]:


# remapping = [0, 1, 2, 3,   4, 5,  6, 7,   8, 9,    11, 10,  15, 14,  13, 12,    16, 17, 19, 18]
rV, rT = sample_tet(6)

ppp = []
ttt = []
tot = 0


for ti in tqdm(range(T.shape[0])):
    TT = T[ti, :]
    pts = V[TT]

    img_p = gmapping(order, rV, pts)

    ppp += img_p.tolist()
    ttt += (rT+tot).tolist()
    tot+=img_p.shape[0]


ppp = np.array(ppp)
ttt = np.array(ttt, dtype=int)

p_uni, _, inverse = np.unique(ppp, return_index=True, return_inverse=True, axis=0)
t_uni = np.array([inverse[ttt[:, 0]], inverse[ttt[:, 1]], inverse[ttt[:, 2]], inverse[ttt[:, 3]]]).transpose()

p = mp.plot(p_uni, t_uni, shading={"wireframe": True})
p.add_points(V, shading={"point_size": 0.05})

In [ ]:
ttt

In [ ]:
x.tolist